In [1]:
# class NaiveBayesC:
#     def __init__(self,alpha = 1):
#         self.alpha = alpha
        
#     def train(self, X, y, fitur):
#         self.X = X
#         self.y = y
#         self.fitur = fitur
#         # print(X)
#         self.len_fitur = len(X.A[0])
# #         self.fitur = fitur
        
#         self.class_ = sorted(set(self.y))

#         self.prior_class = dict(zip(Counter(self.y).keys(), Counter(self.y).values()))
#         for key, value in self.prior_class.items():
#             self.prior_class.update({key:value/len(self.y)})
        
#         #menghitung jumlah class
#         self.len_data = len(self.y)

#         #mencari index data untuk class tertentu
#         self.dic_data_index = dict()
#         for i in self.class_:
#             isi_list = list()
#             for index, j in enumerate(self.y):
#         #         print(j)
#                 if i == j:
#                     isi_list.append(index)
#             self.dic_data_index.update({i:isi_list})
            
#         self.dic_data_by_class = dict()
#         for key, value in self.dic_data_index.items():
#             self.dic_data_by_class.update({key:(self.X[value])})
# #         dic_data_index

#         self.posterior = dict()
#         for c, value in self.dic_data_by_class.items():
#             prob_fitur = list()
#             sum_value  = value.sum()
#             for ix, f in enumerate(value.transpose()):
#                 prob_fitur.append((f.sum()+self.alpha)/(sum_value+self.len_fitur))
#                 print(c,str(f.sum()), "+",str(self.alpha),"/", str(sum_value), str(self.len_fitur), self.fitur[ix])
#             self.posterior.update({c:prob_fitur})
# #         print(self.posterior)

#     def predict(self, X_predict):
#         self.X_predict = X_predict
#         self.inf_dict = dict()
#         for c in self.class_: 
# #             self.sum_predict = dict()
#             self.inf_dict.update({c:np.prod(np.power(self.posterior[c],self.X_predict))*self.prior_class[c]})
#         return max(self.inf_dict.items(), key=operator.itemgetter(1))[0]

In [365]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import operator

class NaiveBayesClassifier:
    def __init__(self, alpha = 1):
        self.dic_data_index = dict()
        self.dic_data_by_class = dict()
        self.dic_data_posterior = dict()
        self.dic_data_prob_fitur = dict()
        self.prior = dict()
        self.class_ = 0
        self.alpha = alpha
#         self.count_doc_c = dict()

    def train (self, X, y):
#         print(X)
        self.X = X
        self.y = y
        
        vectorizer = CountVectorizer(binary=True)
        self.X = vectorizer.fit_transform(X).A
        self.fitur = vectorizer.get_feature_names()
        self.len_fitur = len(self.fitur)
        self.y = np.array(self.y)
        self.class_ = sorted(set(y))
        
        self.prior_class = dict(zip(Counter(self.y).keys(), Counter(self.y).values()))
        self.count_doc_c = self.prior_class.copy()
        for key, value in self.prior_class.items():
#             self.count_doc_c.update({key:len(self.y)})
            self.prior_class.update({key:value/len(self.y)})
        
        #menghitung jumlah class
        self.len_data = len(self.y)

        #mencari index data untuk class tertentu
#         self.dic_data_index = dict()
        for i in self.class_:
            isi_list = list()
            for index, j in enumerate(self.y):
        #         print(j)
                if i == j:
                    isi_list.append(index)
            self.dic_data_index.update({i:isi_list})
            
#         self.dic_data_by_class = dict()
        for key, value in self.dic_data_index.items():
            self.dic_data_by_class.update({key:(self.X[value])})
        del self.dic_data_index
        
        self.count_fitur_c = dict()
        self.dic_data_prob_fitur = dict()
        for c, value in self.dic_data_by_class.items():
            count_per_doc = list()
            prob_fitur = list()
            sum_value  = value.sum()
            for ix, f in enumerate(value.transpose()):
#                 prob_fitur.append((f.sum()+self.alpha)/(len(value)+self.len_fitur))
                count_per_doc.append(f.sum())
                prob_fitur.append((f.sum()/(len(value)+self.len_fitur)))
#                 print(c,str(f.sum()), "+",str(self.alpha),"/", str(len(value)),"+", str(self.len_fitur), self.fitur[ix])
            
            _dict_count_fitur = dict(zip(self.fitur, count_per_doc))
            bobot_fitur = dict(zip(self.fitur,prob_fitur))
            
            self.count_fitur_c.update({c:_dict_count_fitur})
            self.dic_data_prob_fitur.update({c:bobot_fitur})
    
    def predict(self, X):
#         tostr = lamda str(x):x
        if self.alpha <= 0:
            raise Exception("alpha tidak boleh kurang dari atau sama dengan 0, alpha="+str(self.alpha)) 
        X = sorted(set(X.split()))
#         try:
        self.dict_predict = dict()
        for c, value in self.dic_data_prob_fitur.items():
#             print(len(value))

            bobot_fitur = list()
            for kata in X:
#                 print(kata)
                if kata in value:
#                     print(value[kata])
                    # print(c,kata,value[kata])
                    bobot_fitur.append((value[kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur))
                    print("P({}|{})=({}+{})/({}+{})"
                    .format(kata,c,self.count_fitur_c[c][kata],self.alpha,self.count_doc_c[c],self.len_fitur), end=" = ")
                    print((value[kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur))
#  
#                     print("P"+"("+kata+"|"+c+")="+str((value[kata]+(self.alpha/(self.count_doc_c[c]+self.len_fitur)))))
#                     
            if len(bobot_fitur)>0:
                post_prior = np.prod(bobot_fitur)*self.prior_class[c]
#                 print(type(bobot_fitur))
                bobot_fitur_ = [str(round(x, 3)) for x in bobot_fitur]
                print("P({}|{}) = {} x {} = {}".format(c,"X",self.prior_class[c], " x ".join(bobot_fitur_), post_prior,20))
#                 print(bobot_fitur)
                self.dict_predict.update({c:post_prior})
            print("")
        return max(self.dict_predict.items(), key=operator.itemgetter(1))[0]
#         except:
#             print("err404or")
#             return max(self.prior_class.items(), key=operator.itemgetter(1))[0]
            

In [366]:
print("{}+{}".format(1,2))

1+2


In [367]:
import numpy as np
corpus = [
    'Chinese Chinese Shanghai',
    'Chinese Beijing Chinese',
    'Chinese Macao',
    'Tokyo Japan Chinese',
#     'Tokyo Japan',
]

y = np.array(["yes","yes","yes",'no'])

In [368]:
courpus = np.array(corpus)
model = NaiveBayesClassifier()
model.train(corpus, y)

In [369]:
model.alpha = 1
# print(model.alpha)
model.predict("japan tokyo Beijing macao".lower())
# model.prior_class

P(beijing|no)=(0+1)/(1+6) = 0.14285714285714285
P(japan|no)=(1+1)/(1+6) = 0.16326530612244897
P(macao|no)=(0+1)/(1+6) = 0.14285714285714285
P(tokyo|no)=(1+1)/(1+6) = 0.16326530612244897
P(no|X) = 0.25 x 0.143 x 0.163 x 0.143 x 0.163 = 0.00013599775603702534

P(beijing|yes)=(1+1)/(3+6) = 0.1234567901234568
P(japan|yes)=(0+1)/(3+6) = 0.1111111111111111
P(macao|yes)=(1+1)/(3+6) = 0.1234567901234568
P(tokyo|yes)=(0+1)/(3+6) = 0.1111111111111111
P(yes|X) = 0.75 x 0.123 x 0.111 x 0.123 x 0.111 = 0.00014112573173691905



'yes'

In [370]:
model.count_fitur_c

{'no': {'beijing': 0,
  'chinese': 1,
  'japan': 1,
  'macao': 0,
  'shanghai': 0,
  'tokyo': 1},
 'yes': {'beijing': 1,
  'chinese': 3,
  'japan': 0,
  'macao': 1,
  'shanghai': 1,
  'tokyo': 0}}

In [358]:
# model.bobot_fitur

In [357]:
model.dict_predict

{'no': 0.00013599775603702534, 'yes': 0.0001881676423158921}

In [15]:
#no
0.42857142857142855*model.prior_class['no']

0.17142857142857143

In [16]:
#yes
0.2222222222222222*model.prior_class['yes']

0.13333333333333333

In [17]:
model.prior_class#model.prior_class['yes']

{'yes': 0.6, 'no': 0.4}

In [246]:
model.prior_class['no']

0.4

In [200]:
(2+2)/(4+6)

0.4

In [201]:
(2/4)+(2/6)

0.8333333333333333

In [166]:
x= 11
fitur = 50000

In [204]:
(0+x)/(4+fitur)

0.00021998240140788737

In [205]:
0/(4+fitur)+(x/(4+fitur))

0.00021998240140788737

In [209]:
# (2+1)/4

In [210]:
# (2/4)+(1/4)

In [211]:
# (0/4)+(1/4)

In [212]:
# 0/4